In [130]:
import torch
from GCN.data_processing import Data_Loader
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False
from torchvision.models.feature_extraction import create_feature_extractor
import csv
import numpy as np
import io
import os, random
import time
import json
import torch
import zipfile
import numpy as np
import torch.nn as nn
from PIL import Image,ImageOps
import torch.nn.functional as F
from einops import rearrange, repeat
import math
from torch import einsum
from argparse import ArgumentParser
from core.models.curvenet_cls import CurveNet
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import argparse
random_seed = 420 #for reproducibility
from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_error
from torchsummary import summary
import colorama
from colorama import Fore, Back, Style
colorama.init(autoreset=True)
np.seterr(invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [2]:
!nvidia-smi

Mon Sep  4 19:49:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 526.86       Driver Version: 526.86       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |


In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
# print(torch.cuda.current_device())
# print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

|                               |                      |               MIG M. |True

|===============================+======================+======================|1

NVIDIA GeForce GTX 1650|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |

| N/A   68C    P8     1W /  N/A |    367MiB /  4096MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               


In [4]:
# print(len(np.unique(train_y)))
# print(np.unique(train_y))

+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
|        ID   ID                                                   Usage      |
|=============================================================================|
|    0   N/A  N/A      5212    C+G   ...n1h2txyewy\SearchHost.exe    N/A      |
|    0   N/A  N/A      6648    C+G   ...8bbwe\WindowsTerminal.exe    N/A      |
|    0   N/A  N/A      6880    C+G   ...ype for Desktop\Skype.exe    N/A      |

In [5]:
# print(len(train_x))
# print(len(train_y))
# print(len(test_x))
# print(len(test_y))


|    0   N/A  N/A      8204    C+G   C:\Windows\explorer.exe         N/A      |
|    0   N/A  N/A     10012    C+G   ...artMenuExperienceHost.exe    N/A      |
|    0   N/A  N/A     10200    C+G   ...861\Grammarly.Desktop.exe    N/A      |


In [6]:
# train_x.shape

|    0   N/A  N/A     10840    C+G   ...v1g1gvanyjgm\WhatsApp.exe    N/A      |
|    0   N/A  N/A     12664    C+G   ...perience\NVIDIA Share.exe    N/A      |
|    0   N/A  N/A     12936    C+G   ...me\Application\chrome.exe    N/A      |


In [7]:
# train_y[0].shape

|    0   N/A  N/A     13296    C+G   ...2txyewy\TextInputHost.exe    N/A      |
|    0   N/A  N/A     13368    C+G   ...938.69\msedgewebview2.exe    N/A      |
|    0   N/A  N/A     13924    C+G   ...e\PhoneExperienceHost.exe    N/A      |
|    0   N/A  N/A     14592    C+G   ...ray\lghub_system_tray.exe    N/A      |
|    0   N/A  N/A     15144    C+G   ...ype for Desktop\Skype.exe    N/A      |
|    0   N/A  N/A     15840    C+G   ...ation\Adaware-Privacy.exe    N/A      |
|    0   N/A  N/A     16576    C+G   ...938.69\msedgewebview2.exe    N/A      |
|    0   N/A  N/A     17400    C+G   ...y\ShellExperienceHost.exe    N/A      |
|    0   N/A  N/A     18200    C+G   ...wekyb3d8bbwe\Video.UI.exe    N/A      |

In [8]:
# x = torch.tensor(train_x)
# x.size()


|    0   N/A  N/A     18576    C+G   ...938.69\msedgewebview2.exe    N/A      |
|    0   N/A  N/A     20024    C+G   ...938.69\msedgewebview2.exe    N/A      |
|    0   N/A  N/A     23484    C+G   ...ge\Application\msedge.exe    N/A      |
+-----------------------------------------------------------------------------+

In [9]:
class Embedder:
    def __init__(self, **kwargs):
        self.kwargs = kwargs
        self.create_embedding_fn()

    def create_embedding_fn(self):
        embed_fns = []
        d = self.kwargs['input_dims']
        out_dim = 0
        if self.kwargs['include_input']:
            embed_fns.append(lambda x : x)
            out_dim += d

        max_freq = self.kwargs['max_freq_log2']
        N_freqs = self.kwargs['num_freqs']

        if self.kwargs['log_sampling']:
            freq_bands = 2.**torch.linspace(0., max_freq, steps=N_freqs)
        else:
            freq_bands = torch.linspace(2.**0., 2.**max_freq, steps=N_freqs)

        for freq in freq_bands:
            for p_fn in self.kwargs['periodic_fns']:
                embed_fns.append(lambda x, p_fn=p_fn, freq=freq : p_fn(x * freq))
                out_dim += d

        self.embed_fns = embed_fns
        self.out_dim = out_dim

    def embed(self, inputs):
        normalized = torch.cat([fn(inputs) for fn in self.embed_fns], -1)
        return normalized
    
def get_embedder(multires = 10, i=0):
    if i == -1:
        return nn.Identity(), 1

    embed_kwargs = {
                'include_input' : True,
                'input_dims' : 1,
                'max_freq_log2' : multires-1,
                'num_freqs' : multires,
                'log_sampling' : True,
                'periodic_fns' : [torch.sin, torch.cos],
    }

    embedder_obj = Embedder(**embed_kwargs)
    embed = lambda x, eo=embedder_obj : eo.embed(x)
    return embed, embedder_obj.out_dim

embeder = get_embedder()[0]    


In [25]:
class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn

    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)


# GELU -> Gaussian Error Linear Units
class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)


class RemixerBlock(nn.Module):
    def __init__(
        self,
        dim,
        seq_len,
        causal = False,
        bias = False
    ):
        super().__init__()
        self.causal = causal
        self.proj_in = nn.Linear(dim, 2 * dim, bias = bias)
        self.mixer = nn.Parameter(torch.randn(seq_len, seq_len))
        self.alpha = nn.Parameter(torch.tensor(0.))
        self.proj_out = nn.Linear(dim, dim, bias = bias)

    def forward(self, x):
        mixer, causal, device = self.mixer, self.causal, x.device
        x, gate = self.proj_in(x).chunk(2, dim = -1)
        x = F.gelu(gate) * x

        if self.causal:
            seq = x.shape[1]
            mask_value = -torch.finfo(x.dtype).max
            mask = torch.ones((seq, seq), device = device, dtype=torch.bool).triu(1)
            mixer = mixer[:seq, :seq]
            mixer = mixer.masked_fill(mask, mask_value)

        mixer = mixer.softmax(dim = -1)
        mixed = einsum('b n d, m n -> b m d', x, mixer)

        alpha = self.alpha.sigmoid()
        out = (x * mixed) * alpha + (x - mixed) * (1 - alpha)

        return self.proj_out(out)


class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x: torch.FloatTensor) -> torch.FloatTensor:
        """
        Args:
            x: `embeddings`, shape (batch, max_len, d_model)
        Returns:
            `encoder input`, shape (batch, max_len, d_model)
        """
        x = x + self.pe[:, : x.size(1)]
        return self.dropout(x)


class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        
        # print(f'Attention:: {dim} - {heads} - {dim_head} - {dropout}')

        inner_dim = dim_head *  heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.pos_embedding = PositionalEncoding(dim,0.1,128)
        self.attend = nn.Softmax(dim = -1)
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        x += self.pos_embedding(x)
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        attn = self.attend(dots)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)


class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()

        # print('\n')
        # print(f'Transformers:: {dim} - {depth} - {heads} - {dim_head} - {mlp_dim}')

        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout))
                #PreNorm(dim, RemixerBlock(dim,17))
            ]))

    def forward(self, x, swap = False):
        if swap: # for the self.transformer(x,swap = True)
            b, t, n , c = x.size() 
        for idx, (attn, ff) in enumerate(self.layers):
            if swap: # for the self.transformer(x,swap = True)
                if idx % 2 == 0:
                    #* attention along with all timesteps(frames) for each point(landmark)
                    x = rearrange(x, "b t n c -> (b n) t c")
                else:
                    #* attention to all points(landmarks) in each timestep(frame)
                    x = rearrange(x, "b t n c -> (b t) n c")
            x = attn(x) + x  # skip connections
            x = ff(x) + x    # skip connections
            
            # Now return the input x to its original formation
            if swap: # for the self.transformer(x,swap = True)
                if idx % 2 == 0:
                    x = rearrange(x, "(b n) t c -> b t n c", b = b)
                else:
                    x = rearrange(x, "(b t) n c -> b t n c", b = b)
                
        return x


class TemporalModel(nn.Module):
    
    def __init__(self):
        super(TemporalModel,self).__init__()
                
        self.encoder  =  CurveNet() # curve aggregation, needed for Point Clouds Shape Analysis. 
        self.downsample = nn.Sequential(
                            nn.Conv1d(25, 32, kernel_size=1, bias=False),
                            nn.BatchNorm1d(32),
                            # nn.Dropout(p=0.25), #* NEW
                            #nn.ReLU(inplace=True),
                            #nn.Conv1d(128, 32, kernel_size=1, bias=False),
                            #nn.BatchNorm1d(32),
                            )
        
        self.transformer = Transformer(256, 6, 4, 256//4, 256 * 2, 0.1)
        self.time = Transformer(256, 3, 4, 256//4, 256 * 2, 0.1)
        self.dropout = nn.Dropout(0.1)
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(256),
            nn.Linear(256,1),
#             nn.Sigmoid()
        )
        
    def forward(self,x):
        b,t,n,c = x.size()
    
        x = rearrange(x, "b t n c -> (b t) c n")
        x = rearrange(self.dropout(self.encoder(x)), "b c n -> b n c") 
        x = self.downsample(x).view(b,t,32,-1) #b t 32 c
        x = self.transformer(x,swap = True).view(b,t,-1,256).mean(2)
        x = self.time(x).mean(1)
        x = self.mlp_head(x)
        return x

min_xyz = np.array([0.06372425, 0.05751023, -0.08976112]).reshape(1,1,3)
max_xyz = np.array([0.63246971, 1.01475966, 0.14436169]).reshape(1,1,3)


In [11]:
def load_data(path, nr = 80, n_dim= 117, timesteps= 240):
    f = open('./UI-PRMD/'+path+'/Data_Correct.csv')
    csv_f = csv.reader(f)
    Correct_X = list(csv_f)

    # Convert the input sequences into numpy arrays
    train_input1 = np.asarray(Correct_X, dtype = float)
    correct_input = np.zeros((nr,timesteps,n_dim))
    for i in range(len(train_input1)//n_dim):
           correct_input[i,:,:] = np.transpose(train_input1[n_dim*i:n_dim*(i+1),:])
    
    f = open('./UI-PRMD/'+path+'/Labels_Correct.csv')
    csv_f = csv.reader(f)
    Correct_Y = list(csv_f)
    
    # Convert the input labels into numpy arrays
    correct_label = np.asarray(Correct_Y, dtype = float)
    
    f = open('./UI-PRMD/'+path+'/Data_Incorrect.csv')    
    csv_f = csv.reader(f)
    Incorrect_X = list(csv_f)

    # Convert the input sequences into numpy arrays
    test_input1 = np.asarray(Incorrect_X)
    n_dim = 117
    incorrect_input = np.zeros((nr,timesteps,n_dim))
    for i in range(len(test_input1)//n_dim):
          incorrect_input[i,:,:] = np.transpose(test_input1[n_dim*i:n_dim*(i+1),:])
            
    f = open('./UI-PRMD/'+path+'/Labels_Incorrect.csv')
    csv_f = csv.reader(f)
    Incorrect_Y = list(csv_f)
    
    # Convert the input labels into numpy arrays
    incorrect_label = np.asarray(Incorrect_Y, dtype = float)
    
    return correct_input, correct_label, incorrect_input, incorrect_label

In [12]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
def performance_metrics(test_y, y_pred):
    test_dev = abs(test_y-y_pred)
#     mean_abs_dev = torch.mean(test_dev)
    mae = mean_absolute_error(test_y, y_pred)
    rms_dev = sqrt(mean_squared_error(y_pred, test_y))
    mse = mean_squared_error(test_y,y_pred) 
    mape = mean_absolute_percentage_error(test_y, y_pred)
    return mae, rms_dev, mse, mape

In [13]:
def train(epochs,training_generator,test_generator, file, data_loader = None):
    
    con = []      
    net = TemporalModel()
    net.cuda()
    eval_loss = 500
    lr = 0.0001
    optimizer = torch.optim.AdamW(net.parameters(), lr=lr,weight_decay= 0.0)
#     lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[1000], gamma=0.1)
    loss_func = torch.nn.HuberLoss(reduction='mean', delta= 0.1)
    start_time = time.time()
    best_accuracy = 500

    for epoch in range(epochs):
        pred_label = []
        true_label = []
        number_batch = 0
        for x, y in tqdm(training_generator, desc=f"Epoch {epoch}/{epochs-1}", ncols=60):
            if torch.cuda.device_count() > 0:
                x = x.cuda()
                y = y.cuda()
            x = torch.unsqueeze(x, 1) 
#             print(x.size())
            b,d,t,n,c = x.size()
            x = x.view(-1,t,n,c)
            pred = net(x)
            loss = loss_func(pred,y)
            
            
            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            
            pred_label.extend(pred.flatten())
            true_label.extend(y.flatten())
            number_batch += 1
#             lr = lr_scheduler.get_last_lr()[0]

#         lr_scheduler.step()
        
#         print(true_label)
#         print(pred_label)
#         pred_label = torch.cat(pred_label,0)
#         true_label = torch.cat(true_label,0)
#         train_accuracy = torch.sum(pred_label == true_label).type(torch.FloatTensor) / true_label.size(0)
        pred_label = torch.tensor(pred_label)
        true_label = torch.tensor(true_label)
        mae, rms_dev, mse, mape = performance_metrics(true_label, pred_label)
    
        output('Epoch: ' + 'train ' + str(epoch) + 
              ', mae ' + str(mae) + 
              ', rms_dev: ' + str(rms_dev) +
              ', mse: ' + str(mse) +
                ', mape: ' + str(mape)
              )
        print('Epoch: ' + 'train ' + str(epoch) + 
              ', mae ' + str(mae) + 
              ', rms_dev: ' + str(rms_dev) +
              ', mse: ' + str(mse) +
              ', mape: ' + str(mape)
              )
        net.eval()
        
        pred_label = []
        pred_avg   = []
        true_label = []
        with torch.no_grad():
          for x, y in tqdm(test_generator, desc=f"Epoch {epoch}/{epochs-1}", ncols=60):

              if torch.cuda.device_count() > 0:
                  x = x.cuda()
                  y = y.cuda()
              x = torch.unsqueeze(x, 1)
              b,d,t,n,c = x.size()
              x = x.view(-1,t,n,c)
              pred_y    = net(x)
            ###commented to convert round
#               pred_mean = (pred_y.view(b,d).mean(1,keepdim = True) >= 0.5).float().cpu().detach()
#               pred_y    = ((pred_y).view(b,d).mean(1,keepdim = True) >= 0.5).float().cpu().detach()
              
              pred_label.extend(pred_y)
              true_label.extend(y)
# #           pred_avg   = torch.cat(pred_avg,0)  
# #           true_label = torch.cat(true_label,0)
          pred_label = torch.tensor(pred_label)
          true_label = torch.tensor(true_label)
          loss_e = loss_func(pred_label,true_label)
          pred_label = torch.unsqueeze(pred_label,1)
          true_label = torch.unsqueeze(true_label,1)
          
          if data_loader is not None:
              pred_label = data_loader.sc2.inverse_transform(pred_label)
              true_label = data_loader.sc2.inverse_transform(true_label)
                
#           print(pred_label)
#           print(true_label)
          mae, rms_dev, mse, mape = performance_metrics(true_label, pred_label)
#           test_accuracy = torch.sum(pred_label == true_label).type(torch.FloatTensor) / true_label.size(0)
#           test_avg      = torch.sum(pred_avg   == true_label).type(torch.FloatTensor) / true_label.size(0)
          con.append([epoch,mae])
          output('test accuracy: ' + 
              'mae ' + str(mae) + 
              ', rms_dev: ' + str(rms_dev) +
              ', mse: ' + str(mse) +
              ', mape: ' + str(mape) +
                 ', loss' + str(loss_e)
              )
          print(Fore.GREEN + 'test accuracy: ' + 
              'mae ' + str(mae) + 
              ', rms_dev: ' + str(rms_dev) +
              ', mse: ' + str(mse) +
              ', mape: ' + str(mape)+
                ', loss' + str(loss_e)
              )
          print()
          if mae < best_accuracy:
              filepath = f"{file}-{epoch:}-{loss}-{mae}.pt"
              torch.save(net.state_dict(), filepath)
            #   torch.save(net, filepath)
            #   test_frames(f'{test_accuracy}={test_f}')
              output('Best Results Achieved!!!!!!!!!!!!')
              print("Best Results Achieved!!!!!!!!!!!!")
              best_accuracy = mae

        net.train()
        
        output(f"ETA Per Epoch:{(time.time() - start_time) / (epoch + 1)}")
        # print(f"ETA Per Epoch:{(time.time() - start_time) / (epoch + 1)}")

    print("Lowest loss: ", eval_loss)
    
    
# image_size = 48
# label_path = "labels"
# data = "npy"

# sometimes = lambda aug: va.Sometimes(0.5, aug)
# seq = va.Sequential([
#     va.RandomCrop(size=(image_size, image_size)),       
#     sometimes(va.HorizontalFlip()),              
# ])


# label_path = "labels"


In [55]:
def evaluate(test_generator, data_loader, net):
    net.cuda()
    net.eval()
    
    pred_label = []
    pred_avg   = []
    true_label = []
    with torch.no_grad():
      for x, y in tqdm(test_generator, ncols=60):
          if torch.cuda.device_count() > 0:
              x = x.cuda()
              y = y.cuda()
          x = torch.unsqueeze(x, 1)
          b,d,t,n,c = x.size()
          x = x.view(-1,t,n,c)
          pred_y = net(x)
        ###commented to convert round
#               pred_mean = (pred_y.view(b,d).mean(1,keepdim = True) >= 0.5).float().cpu().detach()
#               pred_y    = ((pred_y).view(b,d).mean(1,keepdim = True) >= 0.5).float().cpu().detach()

          pred_label.extend(pred_y)
          true_label.extend(y)
# #           pred_avg   = torch.cat(pred_avg,0)  
# #           true_label = torch.cat(true_label,0)
      pred_label = torch.tensor(pred_label)
      true_label = torch.tensor(true_label)
#       loss_e = loss_func(pred_label,true_label)
      pred_label = torch.unsqueeze(pred_label,1)
      true_label = torch.unsqueeze(true_label,1)

      if data_loader is not None:
          pred_label = data_loader.sc2.inverse_transform(pred_label)
          true_label = data_loader.sc2.inverse_transform(true_label)

#           print(pred_label)
#           print(true_label)
      mae, rms_dev, mse, mape = performance_metrics(true_label, pred_label)
#           test_accuracy = torch.sum(pred_label == true_label).type(torch.FloatTensor) / true_label.size(0)
#           test_avg      = torch.sum(pred_avg   == true_label).type(torch.FloatTensor) / true_label.size(0)
#       con.append([epoch,mae])
      output('test accuracy: ' + 
          'mae ' + str(mae) + 
          ', rms_dev: ' + str(rms_dev) +
          ', mse: ' + str(mse) +
          ', mape: ' + str(mape)
          )
      print(Fore.GREEN + 'test accuracy: ' + 
          'mae ' + str(mae) + 
          ', rms_dev: ' + str(rms_dev) +
          ', mse: ' + str(mse) +
          ', mape: ' + str(mape)
          )

In [102]:
def inference(x, y, net, data_loader):
    net.cuda()
    net.eval()
    x = torch.unsqueeze(x,0)
    y = torch.unsqueeze(y,0)
    pred_label = []
    pred_avg   = []
    true_label = []
    with torch.no_grad():
        x = x.cuda()
        y = y.cuda()
        x = torch.unsqueeze(x, 1)
        b,d,t,n,c = x.size()
        x = x.view(-1,t,n,c)
        pred_y = net(x)
        pred_label.extend(pred_y)
        true_label.extend(y)

        pred_label = torch.tensor(pred_label)
        true_label = torch.tensor(true_label)
#       loss_e = loss_func(pred_label,true_label)
        pred_label = torch.unsqueeze(pred_label,1)
        true_label = torch.unsqueeze(true_label,1)

        if data_loader is not None:
            pred_label = data_loader.sc2.inverse_transform(pred_label)
            true_label = data_loader.sc2.inverse_transform(true_label)
        print("Actual Score: ", true_label)
        print("Predicted Score: ", pred_label)
        mae, rms_dev, mse, mape = performance_metrics(true_label, pred_label)
#           test_accuracy = torch.sum(pred_label == true_label).type(torch.FloatTensor) / true_label.size(0)
#           test_avg      = torch.sum(pred_avg   == true_label).type(torch.FloatTensor) / true_label.size(0)
#       con.append([epoch,mae])
        output('test accuracy: ' + 
          'mae ' + str(mae) + 
          ', rms_dev: ' + str(rms_dev) +
          ', mse: ' + str(mse) +
          ', mape: ' + str(mape)
          )
        print(Fore.GREEN + 'test accuracy: ' + 
          'mae ' + str(mae) + 
          ', rms_dev: ' + str(rms_dev) +
          ', mse: ' + str(mse) +
          ', mape: ' + str(mape)
          )

In [103]:
from sklearn.preprocessing import MinMaxScaler
class prmd_scaler():
    def __init__(self, y_train):
        self.y_train = y_train
        self.sc2 = MinMaxScaler()
        self.scaled_y = self.preprocessing()
    def preprocessing(self):
        y = self.sc2.fit_transform(self.y_train)
        return None

In [128]:
def main():
    
    # Add the arguments
    ####################Parser not working###########################
    ex = "Kimore_ex5"
    lr = 0.0001
    epoch = 2000
    batch_size = 5
    use_type = "viz" #train or test or inference
    ####################Parser not working###########################

    # my_parser.add_argument('--ex', type=str, default="Kimore_ex5",
    #                        help='the name of exercise.', required=True)
    # my_parser.add_argument('--lr', type=int, default= 0.0001,
    #                        help='initial learning rate for optimizer.')
    # my_parser.add_argument('--epoch', type=int, default= 1000,
    #                        help='number of epochs to train.')
    # my_parser.add_argument('--batch_size', type=int, default= 10,
    #                        help='training batch size.')
    # args = my_parser.parse_args()
    global output
    def output(s):
        with open(f"log_{ex}","a") as f:
            f.write(str(s) + "\n")
            
    if "Kimore" in ex:
        data_loader = Data_Loader(ex) 
        train_x, test_x, train_y, test_y = train_test_split(data_loader.scaled_x, data_loader.scaled_y, test_size=0.2, 
                                                            random_state = random_seed)
        
    else:
        nr = 80
        Correct_data, Correct_label, Incorrect_data, Incorrect_label = load_data(ex)
        print(Correct_data.shape[0])
        print(Correct_label.shape)
        trainidx1 = random.sample(range(0,Correct_data.shape[0]),int(nr*0.8))
        trainidx2 = random.sample(range(0,Incorrect_data.shape[0]),int(nr*0.8))
        valididx1 = np.setdiff1d(np.arange(0,nr,1),trainidx1)
        valididx2 = np.setdiff1d(np.arange(0,nr,1),trainidx2)
        # Training set: data and labels
        train_x = np.concatenate((Correct_data[trainidx1,:,:],Incorrect_data[trainidx2,:,:]))
        train_y = np.concatenate((np.squeeze(Correct_label[trainidx1]),np.squeeze(Incorrect_label[trainidx2])))

        # Validation set: data and labels
        test_x = np.concatenate((Correct_data[valididx1,:,:],Incorrect_data[valididx2,:,:]))
#         print(valid_x.shape, 'validation data')
        test_y = np.concatenate((np.squeeze(Correct_label[valididx1]),np.squeeze(Incorrect_label[valididx2])))
        
#         print(valid_y.shape, 'validation labels')
        
    
    
    print(train_x.shape)
    print(train_y.shape)
    print(test_x.shape)
    print(test_y.shape)
    train_x = torch.Tensor(train_x)
    train_y = torch.Tensor(train_y)
    test_x  = torch.Tensor(test_x)
    test_y = torch.Tensor(test_y)
    if "UI" in ex:
        train_x = torch.unsqueeze(train_x, 3)
        train_y = torch.unsqueeze(train_y, 1)
        train_x = train_x.expand(train_x.size(dim = 0),train_x.size(dim = 1),train_x.size(dim = 2),3)
        print(train_x.size())
        test_x = torch.unsqueeze(test_x,3)
        test_y = torch.unsqueeze(test_y, 1)
        test_x= test_x.expand(test_x.size(dim = 0), test_x.size(dim = 1), test_x.size(dim = 2),3)
        print(test_x.size())
#         data_loader = prmd_scaler(train_y)
#         train_y = data_loader.scaled_y
# #         test_y = data_loader.scaled_yt
# #         print(test_y[:10])
#         train_y = torch.Tensor(train_y)
#         test_y = torch.Tensor(test_y)
        print(train_y.size())
        print(test_y.size())
        
        
    dataset = torch.utils.data.TensorDataset(train_x, train_y)
    training_generator = torch.utils.data.DataLoader(dataset, batch_size = batch_size, shuffle = True)
    dataset = torch.utils.data.TensorDataset(test_x, test_y)
    test_generator = torch.utils.data.DataLoader(dataset, batch_size = batch_size, shuffle = False)
    
    if use_type =='train':
        current_path = './models_UI/'
        train(epoch,training_generator,test_generator,current_path)
    elif use_type == 'test':
        model_directory = ".\models_KIMORE\models_KIMORE\Kimore_ex5.pt"
        net = TemporalModel()
        net.load_state_dict(torch.load(model_directory))
        print(net)
        print(torch.cuda.device_count())
#         evaluate(test_generator, data_loader, net)
#         print(dataset[0][0])
        print(dataset[2][0].shape)
    elif use_type == 'inference':
        model_directory = ".\models_KIMORE\models_KIMORE\Kimore_ex5.pt"
        net = TemporalModel()
        net.load_state_dict(torch.load(model_directory))        
        inference(dataset[5][0], dataset[5][1], net, data_loader)
    else:
        model_directory = ".\models_KIMORE\models_KIMORE\Kimore_ex5.pt"
        net = TemporalModel()
        net.load_state_dict(torch.load(model_directory))
        print(net.transformer)
        return_nodes = {
    "transformer": "transformer"
}
        
        model2 = create_feature_extractor(net, return_nodes=return_nodes)
        intermediate_outputs = model2(dataset[5][0])
        print(intermediate_outputs.shape)


In [129]:
if __name__ == "__main__":
    main()

(298, 100, 25, 3)
(298, 1)
(75, 100, 25, 3)
(75, 1)
Transformer(
  (layers): ModuleList(
    (0): ModuleList(
      (0): PreNorm(
        (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (fn): Attention(
          (pos_embedding): PositionalEncoding(
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (attend): Softmax(dim=-1)
          (to_qkv): Linear(in_features=256, out_features=768, bias=False)
          (to_out): Sequential(
            (0): Linear(in_features=256, out_features=256, bias=True)
            (1): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (1): PreNorm(
        (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (fn): FeedForward(
          (net): Sequential(
            (0): Linear(in_features=256, out_features=512, bias=True)
            (1): GELU()
            (2): Dropout(p=0.1, inplace=False)
            (3): Linear(in_features=512, out_features=256, bias=True)
            (4): 

In [123]:
if __name__ == "__main__":
    main()

(298, 100, 25, 3)
(298, 1)
(75, 100, 25, 3)
(75, 1)
TemporalModel(
  (encoder): CurveNet(
    (lpfa): LPFA(
      (mlp): Sequential(
        (0): Sequential(
          (0): Conv2d(9, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): LeakyReLU(negative_slope=0.2)
        )
      )
    )
    (cic11): CIC(
      (curveaggregation): CurveAggregation(
        (conva): Conv1d(16, 8, kernel_size=(1,), stride=(1,), bias=False)
        (convb): Conv1d(16, 8, kernel_size=(1,), stride=(1,), bias=False)
        (convc): Conv1d(16, 8, kernel_size=(1,), stride=(1,), bias=False)
        (convn): Conv1d(8, 8, kernel_size=(1,), stride=(1,), bias=False)
        (convl): Conv1d(8, 8, kernel_size=(1,), stride=(1,), bias=False)
        (convd): Sequential(
          (0): Conv1d(16, 16, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, t

)
1
tensor([0.4481])


In [ ]:

# Print the size of the data 
print(Correct_data.shape, 'correct sequences')
print(Correct_label.shape, 'correct labels')
print(Incorrect_data.shape, 'incorrect sequences')
print(Incorrect_label.shape, 'incorrect labels')